# Hydrology Lab: Modified Rational Method – Discrete Convolution

This lab explores the **Modified Rational Method** using **discrete convolution** to estimate runoff hydrographs. You will learn how to:

- Generate rainfall hyetographs
- Compute direct runoff using convolution
- Plot hydrographs and interpret peak flow and runoff volume
- Compare results under different storm types and watershed parameters

**Date**: April 04, 2025

---

**Learning Objectives**:
- Understand the physical meaning of the Rational Method and its convolution form
- Interpret and manipulate SCS design storms
- Analyze hydrograph results and estimate peak discharge and runoff volume


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
# User Input Parameters
PT = 6.96  # Total storm depth in inches
area = 181  # Drainage area in acres
C = 0.65  # Runoff coefficient
Tc = 66.0  # Time of concentration in minutes
hyetType = 'type2'  # Options: 'type1', 'type1A', 'type2', 'type3', 'user'


In [ ]:
def get_hyetograph(hyetType, PT):
    hour = [0,2,4,6,7,8,8.5,9,9.5,9.75,10,10.5,11,11.5,11.75,12,12.5,13.0,13.6,14,16,20,24,48]
    minutes = [i*60 for i in hour]
    hyets = {
        'type1': [0,0.035,0.076,0.125,0.156,0.194,0.219,0.254,0.303,0.362,0.515,0.583,0.624,0.654,0.669,0.682,0.706,0.727,0.748,0.767,0.83,0.926,1,1],
        'type1A': [0,0.05,0.116,0.206,0.268,0.425,0.48,0.52,0.55,0.564,0.577,0.601,0.624,0.645,0.655,0.664,0.683,0.701,0.719,0.736,0.8,0.906,1,1],
        'type2': [0,0.022,0.048,0.08,0.098,0.12,0.133,0.147,0.163,0.172,0.181,0.204,0.235,0.283,0.357,0.663,0.735,0.772,0.799,0.82,0.88,0.952,1,1],
        'type3': [0,0.02,0.043,0.072,0.089,0.115,0.13,0.148,0.167,0.178,0.189,0.216,0.25,0.298,0.339,0.5,0.702,0.751,0.785,0.811,0.886,0.957,1,1],
        'user': [0,0,0.4285,0.8571,1.0,1.0,1.0,1.0]
    }
    if hyetType == 'user':
        user_time = [0,7,8,9,9.3333,10,24,48]
        minutes = [i*60 for i in user_time]
        hyet = hyets['user']
    else:
        hyet = hyets.get(hyetType)
    f = interp1d(minutes, hyet, kind='linear')
    t24 = np.arange(0, 2881)
    depth = PT * f(t24)
    intensity = np.diff(np.insert(depth, 0, 0)) * 60
    return t24, intensity, depth


In [ ]:
def kernel(time, area, Tc):
    return area/Tc if time <= Tc else 0.0

def Convolve(excitation, kernel):
    duration = len(excitation)
    response = np.zeros(duration)
    for i in range(duration):
        for j in range(i, duration-1):
            response[j] += excitation[i] * kernel[(j - i) + 1]
    return response


In [ ]:
def ModRat(area, C, precipitation, time, Tc):
    excess = C * np.array(precipitation)
    unitgraph = np.array([kernel(t, area, Tc) for t in time])
    unitgraph /= unitgraph.sum()
    flow = Convolve(excess, unitgraph)
    return area * flow


In [ ]:
t24, r24, d24 = get_hyetograph(hyetType, PT)
result = ModRat(area, C, r24, t24, Tc)
peakQ = max(result)
totalQ = np.sum(result) * 60 / 43560
totalR = C * d24[-1] * (1/12) * area
massE = (totalR - totalQ) / totalR


In [ ]:
def plot_hydrograph(t24, r24, result, PT, C, Tc, area, hyetType, peakQ, totalQ, totalR, massE):
    labels = {'type1': 'SCS Type 1', 'type1A': 'SCS Type 1A', 'type2': 'SCS Type 2',
              'type3': 'SCS Type 3', 'user': 'User Defined'}
    desc = labels.get(hyetType, 'Unknown')
    title = (
        f"Modified Rational Method – Hydrograph\n\n"
        f"Hyetograph: {desc} | Area: {area:.1f} ac | Depth: {PT:.2f} in | C: {C:.2f} | Tc: {Tc:.1f} min\n"
        f"Peak Q: {peakQ:.1f} cfs | Q total: {totalQ:.2f} af | R total: {totalR:.2f} af | Error: {massE*100:.2f}%"
    )
    fig, ax1 = plt.subplots(figsize=(10, 6))
    ax1.step(t24, r24, color='blue', label='Rainfall Intensity (in/hr)')
    ax1.set_ylabel('Rainfall Intensity (in/hr)', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.set_xlabel('Time (minutes)')
    ax1.grid(True)

    ax2 = ax1.twinx()
    ax2.step(t24, result, color='red', label='Runoff Discharge (cfs)')
    ax2.set_ylabel('Runoff Discharge (cfs)', color='red')
    ax2.tick_params(axis='y', labelcolor='red')

    fig.suptitle(title, fontsize=10, ha='left', x=0.01, wrap=True)
    fig.tight_layout(rect=[0, 0, 1, 0.93])
    plt.show()

plot_hydrograph(t24, r24, result, PT, C, Tc, area, hyetType, peakQ, totalQ, totalR, massE)


---

### 🔍 Reflection and Questions

1. **How does the time of concentration affect the peak discharge and the shape of the hydrograph?**

2. **Compare two storm types using the same watershed parameters (change `hyetType`) and observe how peak flow changes.**

3. **Explain the effect of runoff coefficient `C` on both the timing and magnitude of runoff.**

4. **Suppose this watershed were partially developed, how would you modify this model to account for changes in impervious area?**
